In [64]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from stable_baselines import TRPO
from gym import Env 
from gym.spaces import MultiBinary, Box
import numpy as np
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env
import gym

In [65]:
# Create custom environment 
class Mario(Env): 
    def __init__(self):
        super(Mario, self).__init__()
        self.observation_space = Box(low=0, high=255, shape=(240, 256, 3), dtype=np.uint8)
        self.action_space = MultiBinary(7)
        # Startup and instance of the game 
        self._current_score = 0
        # self.env = gym_super_mario_bros.make('SuperMarioBros-v0')
        self.env = gym_super_mario_bros.make('SuperMarioBros-1-4-v0')



    def step(self, action):
        state, reward, done, info = self.env.step(action)
        reward += (info['score'] - self._current_score) / 40.0
        self._current_score = info['score']
        if done:
            if info['flag_get']:
                print('We got it!!!!!')
                reward += 350.0
            else:
                reward -= 50.0
        return state, reward / 10.0, done, info

    def reset(self):
        """Reset the environment and return the initial observation."""
        return self.env.reset()
    
    def render(self, *args, **kwargs):
        self.env.render()
        
    def close(self):
        self.env.close()
    

In [66]:
env=Mario()
env = JoypadSpace(env, SIMPLE_MOVEMENT)


monitor_dir = './logs/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')

# Set Callback

In [67]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, save_model_dir, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = os.path.join(save_model_dir, 'best_model/')
        self.best_mean_reward = -np.inf

    # def _init_callback(self) -> None:
    def _init_callback(self):
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    # def _on_step(self) -> bool:
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            print('self.n_calls: ',self.n_calls)
            model_path1 = os.path.join(self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path1)

        return True

In [68]:
save_model_dir = './train/'
callback1 = SaveOnBestTrainingRewardCallback(100000, save_model_dir)

# Training

In [69]:
model_param_1={
 #    'n_steps': 7149,
 # 'gamma': 0.8692871366327747,
 # 'learning_rate': 6.442559213980066e-05,
 # 'clip_range': 0.31688308594665404,
 # 'gae_lambda': 0.8710254680014865
}

In [70]:
tensorboard_log = r'./tensorboard_log/'

model = TRPO("CnnPolicy", env, verbose=1,
            tensorboard_log = monitor_dir,**model_param_1)
model.learn(total_timesteps=4000000,callback=callback1)
model.save("mario_model")

********** Iteration 0 ************
Optimizing Policy...
sampling


ValueError: Cannot feed value of shape (1, 1, 240, 256, 4) for Tensor 'input/Ob:0', which has shape '(?, 240, 256, 4)'

# Continue Training

In [ ]:
# tensorboard_log = r'./tensorboard_log/'
#
# model = PPO("CnnPolicy", env, verbose=1,
#             tensorboard_log = monitor_dir,**model_param_1)

In [ ]:
# model.set_parameters(r'C:\Users\admin\Desktop\NZH\Mario Code\train\best_model\model_3500000.zip')

In [ ]:
# model.learn(total_timesteps=4000000,callback=callback1)

# Testing

In [ ]:
model = PPO.load("mario_model")

obs = env.reset()
obs=obs.copy()
done = True
while True:
    if done:
        state = env.reset()
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    print(info['score'])
    obs=obs.copy()
    env.render()

In [ ]:
env.close()